## 前言
本项目基于 2020-9-22 采集的厦门在售二手房数据，目的是构建一个简易的二手房价格预测模型。因为模型最后输出的是具体数值，所以项目中的模型属于回归模型。项目包括6个步骤：数据清洗、特征选择、数据转换、划分训练集和测试集、构建模型、模型优化。

In [1]:
import numpy as np
import re
import warnings
import pandas as pd
pd.set_option('display.max_columns', None)  # 显示所有列
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  # 图表可以显示中文
plt.rcParams['axes.unicode_minus'] = False    # 图表可以显示负号
import seaborn as sns
warnings.filterwarnings('ignore')
from sklearn.feature_extraction import DictVectorizer  # 实现独热编码
from sklearn.model_selection import train_test_split,GridSearchCV  # 拆分数据集，模型调优
from sklearn.tree import DecisionTreeRegressor  # 决策树回归
from sklearn.neighbors import KNeighborsRegressor  # KNN回归
from sklearn.linear_model import LinearRegression  # 线性回归
from sklearn.ensemble import AdaBoostRegressor  # AdaBoost回归
from sklearn.metrics import mean_absolute_error  # 平均绝对误差
from sklearn.metrics import mean_squared_error  # 均方误差
from sklearn.metrics import r2_score  # 拟合度
from sklearn.metrics import make_scorer  # 积分函数
from sklearn.model_selection import KFold  # 交叉验证


Bad key "text.kerning_factor" on line 4 in
D:\Program Files (x86)\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## 数据概览

In [2]:
# 在售房源数据
df = pd.read_excel(r"厦门链家网在售二手房爬虫数据_2020-09-22.xlsx",dtype={'房源编码':str})
df.head(3)

,Unnamed: 0,房源链接,总价,总价单位,单价,小区名称,所在大区,所在详细区域,房屋户型,所在楼层,建筑面积,户型结构,套内面积,建筑类型,房屋朝向,建筑结构,装修情况,梯户比例,配备电梯,挂牌时间,交易权属,上次交易,房屋用途,房屋年限,产权所属,抵押信息,房本备件,房源编码,用水类型,用电类型,燃气价格,别墅类型
0,0,https://xm.lianjia.com/ershoufang/105104628729...,660.0,万,51985元/平米,玉成豪园,思明,莲前,3室1厅1厨2卫,低楼层 (共19层),126.96㎡,平层,暂无数据,板楼,东南,钢混结构,精装,两梯两户,有,2020-06-18,商品房,2020-06-20,普通住宅,未满两年,共有,\n 有抵押 30万元 建设银...,已上传房本照片,00381041,NaN,NaN,NaN,NaN
1,1,https://xm.lianjia.com/ershoufang/105104643967...,789.7,万,53000元/平米,海豚湾,思明,会展中心,3室2厅1厨2卫,低楼层 (共32层),149㎡,错层,暂无数据,塔楼,东南,框架结构,精装,两梯两户,有,2020-06-20,商品房,暂无数据,普通住宅,暂无数据,共有,\n 无抵押\n ...,未上传房本照片,NaN,NaN,NaN,NaN,NaN
2,2,https://xm.lianjia.com/ershoufang/105104685921...,1080.0,万,80796元/平米,蓝湾国际,思明,体育中心,3室2厅1厨1卫,低楼层 (共31层),133.67㎡,平层,暂无数据,板楼,东南,钢混结构,精装,三梯三户,有,2020-06-26,商品房,暂无数据,普通住宅,暂无数据,非共有,\n 无抵押\n ...,未上传房本照片,00248646,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16219 entries, 0 to 16218
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  16219 non-null  int64  
 1   房源链接        16219 non-null  object 
 2   总价          16219 non-null  float64
 3   总价单位        16219 non-null  object 
 4   单价          16219 non-null  object 
 5   小区名称        16219 non-null  object 
 6   所在大区        16219 non-null  object 
 7   所在详细区域      16218 non-null  object 
 8   房屋户型        15753 non-null  object 
 9   所在楼层        16219 non-null  object 
 10  建筑面积        16219 non-null  object 
 11  户型结构        15616 non-null  object 
 12  套内面积        15753 non-null  object 
 13  建筑类型        15616 non-null  object 
 14  房屋朝向        16219 non-null  object 
 15  建筑结构        15753 non-null  object 
 16  装修情况        15753 non-null  object 
 17  梯户比例        15616 non-null  object 
 18  配备电梯        15616 non-null  object 
 19  挂牌时间        16219 non-nul

## 数据清洗

#### 重复值检查

In [4]:
print(len(df[df.duplicated()])) # 查看重复值数量
# df.drop_duplicates(inplace=True)  # 删除重复值

0


In [5]:
# 检查总价单位是否统一
df['总价单位'].value_counts()

万    16219
Name: 总价单位, dtype: int64

#### 检查空值

In [6]:
df.columns

Index(['Unnamed: 0', '房源链接', '总价', '总价单位', '单价', '小区名称', '所在大区', '所在详细区域',
       '房屋户型', '所在楼层', '建筑面积', '户型结构', '套内面积', '建筑类型', '房屋朝向', '建筑结构', '装修情况',
       '梯户比例', '配备电梯', '挂牌时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息',
       '房本备件', '房源编码', '用水类型', '用电类型', '燃气价格', '别墅类型'],
      dtype='object')

In [7]:
# 把暂无数据的值替换为空值，原先空值先不动
for column in df.columns:
    df[column][df[column] == '暂无数据'] = np.nan

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16219 entries, 0 to 16218
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  16219 non-null  int64  
 1   房源链接        16219 non-null  object 
 2   总价          16219 non-null  float64
 3   总价单位        16219 non-null  object 
 4   单价          16219 non-null  object 
 5   小区名称        16219 non-null  object 
 6   所在大区        16219 non-null  object 
 7   所在详细区域      16218 non-null  object 
 8   房屋户型        15753 non-null  object 
 9   所在楼层        16219 non-null  object 
 10  建筑面积        16219 non-null  object 
 11  户型结构        14991 non-null  object 
 12  套内面积        1697 non-null   object 
 13  建筑类型        15183 non-null  object 
 14  房屋朝向        16219 non-null  object 
 15  建筑结构        15753 non-null  object 
 16  装修情况        15753 non-null  object 
 17  梯户比例        15616 non-null  object 
 18  配备电梯        14973 non-null  object 
 19  挂牌时间        16219 non-nul

In [9]:
df.isnull().any()

Unnamed: 0    False
房源链接          False
总价            False
总价单位          False
单价            False
小区名称          False
所在大区          False
所在详细区域         True
房屋户型           True
所在楼层          False
建筑面积          False
户型结构           True
套内面积           True
建筑类型           True
房屋朝向          False
建筑结构           True
装修情况           True
梯户比例           True
配备电梯           True
挂牌时间          False
交易权属          False
上次交易           True
房屋用途          False
房屋年限           True
产权所属           True
抵押信息          False
房本备件          False
房源编码           True
用水类型           True
用电类型           True
燃气价格           True
别墅类型           True
dtype: bool

空值特征比较多，需要分别进行处理。

#### 所在详细区域特征清洗

In [10]:
df[df['所在详细区域'].isnull()]

,Unnamed: 0,房源链接,总价,总价单位,单价,小区名称,所在大区,所在详细区域,房屋户型,所在楼层,建筑面积,户型结构,套内面积,建筑类型,房屋朝向,建筑结构,装修情况,梯户比例,配备电梯,挂牌时间,交易权属,上次交易,房屋用途,房屋年限,产权所属,抵押信息,房本备件,房源编码,用水类型,用电类型,燃气价格,别墅类型
5857,5857,https://xm.lianjia.com/ershoufang/105104214320...,316.0,万,16142元/平米,东坂花园,海沧,NaN,4室2厅1厨3卫,高楼层 (共10层),195.77㎡,复式,NaN,NaN,南,未知结构,精装,一梯一户,有,2020-04-28,动迁安置房,2018-03-15,普通住宅,满两年,非共有,\n 有抵押 90万元 中国银...,已上传房本照片,00517931,NaN,NaN,NaN,NaN


根据小区所在的区域位于翔安东孚，我们把空值填充为'东孚'

In [11]:
df['所在详细区域'].fillna('东孚',inplace=True)

In [12]:
df['所在详细区域'].value_counts()

新店       1334
翔安新城      813
集美其它      759
同安其它      629
锦园        551
         ... 
黄厝          4
曾厝垵         4
外国语学校       4
软件园         2
海沧农场        2
Name: 所在详细区域, Length: 83, dtype: int64

#### 房屋户型特征清洗

In [13]:
df['房屋户型'].value_counts()

3室2厅1厨2卫    3328
2室2厅1厨1卫    2865
2室1厅1厨1卫    2372
3室2厅1厨1卫    1838
4室2厅1厨2卫    1376
            ... 
5室4厅1厨5卫       1
9室3厅1厨7卫       1
8室2厅2厨4卫       1
6室4厅2厨6卫       1
6室1厅1厨2卫       1
Name: 房屋户型, Length: 161, dtype: int64

房屋户型特征的格式类似'3室2厅1厨2卫'，我们采用分列的方式来处理，新建'室'、'厅'、'厨'、'卫'4列，分别用来保存户型数据。

In [14]:
df['房屋户型'].fillna('0室0厅0厨0卫', inplace=True)
find1 = re.compile('(\d*)室(\d*)厅(\d*)厨(\d*)卫')
df['室'] = df['房屋户型'].map(lambda x: int(find1.findall(x)[0][0]))
df['厅'] = df['房屋户型'].map(lambda x: int(find1.findall(x)[0][1]))
df['厨'] = df['房屋户型'].map(lambda x: int(find1.findall(x)[0][2]))
df['卫'] = df['房屋户型'].map(lambda x: int(find1.findall(x)[0][3]))

#### 所在楼层特征清洗

In [15]:
df['所在楼层'].value_counts()

低楼层 (共18层)    635
中楼层 (共7层)     621
高楼层 (共7层)     571
中楼层 (共18层)    540
高楼层 (共18层)    479
             ... 
中楼层 (共55层)      2
低楼层 (共55层)      2
高楼层 (共48层)      1
地下室 (共33层)      1
低楼层 (共52层)      1
Name: 所在楼层, Length: 161, dtype: int64

所在楼层字段，拆分成2个字段，所在楼层字段只保留‘低楼层’、‘中楼层’等信息，新建‘总楼层’字段用来保存层数。

In [16]:
find2 = re.compile('\d{1,3}')
df['总楼层'] = df['所在楼层'].map(lambda x: int(find2.findall(x)[0]))
df['所在楼层'] = df['所在楼层'].map(lambda x: x.split(' ')[0])

#### 单价、建筑面积特征清洗

In [17]:
df['单价']=df['单价'].str.replace('元/平米','').astype(np.float64)
df['建筑面积']=df['建筑面积'].str.replace('㎡','').astype(np.float64)

#### 户型结构特征清洗

In [18]:
df['户型结构'].value_counts()

平层    13487
复式     1081
错层      217
跃层      206
Name: 户型结构, dtype: int64

In [19]:
df['户型结构'].fillna('平层',inplace=True)

#### 建筑类型特征清洗

In [20]:
df['建筑类型'].value_counts()

板楼      7871
塔楼      3817
板塔结合    3377
平房       118
Name: 建筑类型, dtype: int64

In [21]:
df['建筑类型'].fillna('板楼',inplace=True)

#### 房屋朝向特征清洗

In [22]:
df['房屋朝向'].value_counts()

南              5330
南 北            4204
东南             2988
北               545
西南              531
               ... 
东 东南 西            1
东南 东 南 西南 西       1
西 东 南             1
东 西 北 东北          1
南 西北 北 东北         1
Name: 房屋朝向, Length: 117, dtype: int64

房屋朝向格式模糊不规范，我们统一以第一方位为准，假如一个房屋的朝向是'东南 南 北 东北'，那我们以第一方位'东南'为准。

In [23]:
df['房屋朝向']=df['房屋朝向'].map(lambda x: x.split(' ')[0])
df['房屋朝向'].value_counts()

南     9849
东南    3448
东      982
北      685
西南     616
西      270
西北     189
东北     180
Name: 房屋朝向, dtype: int64

#### 建筑结构特征清洗

In [24]:
df['建筑结构'].value_counts()

钢混结构    12213
框架结构     1872
砖混结构      651
混合结构      509
未知结构      414
钢结构        91
砖木结构        3
Name: 建筑结构, dtype: int64

In [25]:
df['建筑结构'].fillna('钢混结构',inplace=True)

#### 装修情况特征清洗

In [26]:
df['装修情况'].value_counts()

精装    8870
简装    4662
毛坯    2135
其他      86
Name: 装修情况, dtype: int64

In [27]:
df['装修情况'].fillna('精装',inplace=True)

#### 配备电梯特征清洗

配备电梯这列有空值，进行填充，采取的方法是，根据楼层来判断，一般的楼层大于6的都有电梯，而小于等于6层的一般都没有电梯，所以大于6层的空值填充为“有”，小于6层的空值填充为“无”。

In [28]:
df.loc[(df['配备电梯'].isnull())&(df['总楼层']>6),'配备电梯']='有'
df.loc[(df['配备电梯'].isnull())&(df['总楼层']<=6),'配备电梯']='无'
df['配备电梯'].value_counts()

有    12361
无     3858
Name: 配备电梯, dtype: int64

#### 房屋年限特征清洗

In [29]:
df['房屋年限'].value_counts()

满五年     3178
满两年     3033
未满两年    1039
Name: 房屋年限, dtype: int64

'房屋年限'字段划分不够细致，所以我们用挂牌时间和上次交易时间的差值来重新计算具体年限。

In [30]:
# 转化为日期格式
df['挂牌时间'] = pd.to_datetime(df['挂牌时间'])
df['上次交易'] = pd.to_datetime(df['上次交易'])
df['房屋年限/年'] = df['挂牌时间']-df['上次交易']  # 得到的单位是：天
df['房屋年限/年'] = df['房屋年限/年'].map(lambda x: x.days)  # 去掉单位
df['房屋年限/年']=df['房屋年限/年'].map(lambda x: '%.1f'%(x/365))  # 转化为年，保留一位小数
df['房屋年限/年']=df['房屋年限/年'].astype(np.float64)  # 避免下一步比较失败
df['房屋年限/年'][df['房屋年限/年'] <= 0] = 0.1  # 小于0的用0.1表示，便于分箱
df['房屋年限/年'].value_counts()

0.1     659
3.0     226
4.0     187
2.0     168
2.2     123
       ... 
30.2      1
23.5      1
21.7      1
24.5      1
30.3      1
Name: 房屋年限/年, Length: 257, dtype: int64

'房屋年限/年'这个特征是连续型变量，如果每个年份都作为特征值，并不能找出字段对价格的影响，因为年限划分太细，所以我们将这个连续型数值特征离散化，进行分箱处理。另外，房屋年限/年 特征缺失率为55%，无法补齐，所以空值不填充。

In [31]:
# 指定区间
print('房屋年限最长是{}年'.format(df['房屋年限/年'].max()))
bins = [x for x in range(37)]  # 每隔一年一个区间
# labels = [str(x) for x in range(1, 37)]
df['房屋年限/年'] = pd.cut(df['房屋年限/年'], bins=bins)
df['房屋年限/年'] = df['房屋年限/年'].astype(str)
df['房屋年限/年'].value_counts()

房屋年限最长是36.0年


nan             8949
(2.0, 3.0]      1100
(0.0, 1.0]      1032
(3.0, 4.0]      1002
(1.0, 2.0]       698
(4.0, 5.0]       685
(5.0, 6.0]       489
(9.0, 10.0]      322
(6.0, 7.0]       317
(7.0, 8.0]       253
(10.0, 11.0]     225
(8.0, 9.0]       223
(11.0, 12.0]     151
(12.0, 13.0]     130
(13.0, 14.0]     122
(15.0, 16.0]     107
(14.0, 15.0]      92
(16.0, 17.0]      83
(17.0, 18.0]      68
(19.0, 20.0]      40
(20.0, 21.0]      38
(18.0, 19.0]      25
(21.0, 22.0]      25
(24.0, 25.0]      10
(22.0, 23.0]      10
(23.0, 24.0]       7
(30.0, 31.0]       4
(27.0, 28.0]       4
(25.0, 26.0]       2
(33.0, 34.0]       1
(31.0, 32.0]       1
(35.0, 36.0]       1
(29.0, 30.0]       1
(28.0, 29.0]       1
(26.0, 27.0]       1
Name: 房屋年限/年, dtype: int64

#### 房屋用途特征清洗

In [32]:
df['房屋用途'].value_counts()

普通住宅     15054
商业办公类      561
车库         466
别墅         137
平房           1
Name: 房屋用途, dtype: int64

In [33]:
df[df['房屋用途']=='平房']

,Unnamed: 0,房源链接,总价,总价单位,单价,小区名称,所在大区,所在详细区域,房屋户型,所在楼层,建筑面积,户型结构,套内面积,建筑类型,房屋朝向,建筑结构,装修情况,梯户比例,配备电梯,挂牌时间,交易权属,上次交易,房屋用途,房屋年限,产权所属,抵押信息,房本备件,房源编码,用水类型,用电类型,燃气价格,别墅类型,室,厅,厨,卫,总楼层,房屋年限/年
1117,1117,https://xm.lianjia.com/ershoufang/105103433264...,1800.0,万,79827.0,万科湖心岛,湖里,湖边水库,5室2厅1厨3卫,中楼层,225.49,平层,NaN,板塔结合,西南,钢混结构,毛坯,一梯四户,有,2019-11-26,商品房,NaT,平房,NaN,共有,\n 有抵押 500万元 华夏...,未上传房本照片,00017967,NaN,NaN,NaN,NaN,5,2,1,3,5,nan


房屋用途的平房数据只有一条，怀疑数据录入有误，通过观察这条数据对应的小区名称，根据梯户比例字段，我们把这个房屋用途改为普通住宅

In [34]:
df['房屋用途'][df['房屋用途']=='平房']='普通住宅'

#### 产权所属特征清洗

In [35]:
df['产权所属'].value_counts()

非共有    11669
共有      4517
Name: 产权所属, dtype: int64

In [36]:
df['产权所属'].fillna('非共有',inplace=True)

In [37]:
df.isnull().any()

Unnamed: 0    False
房源链接          False
总价            False
总价单位          False
单价            False
小区名称          False
所在大区          False
所在详细区域        False
房屋户型          False
所在楼层          False
建筑面积          False
户型结构          False
套内面积           True
建筑类型          False
房屋朝向          False
建筑结构          False
装修情况          False
梯户比例           True
配备电梯          False
挂牌时间          False
交易权属          False
上次交易           True
房屋用途          False
房屋年限           True
产权所属          False
抵押信息          False
房本备件          False
房源编码           True
用水类型           True
用电类型           True
燃气价格           True
别墅类型           True
室             False
厅             False
厨             False
卫             False
总楼层           False
房屋年限/年        False
dtype: bool

## 特征选择

In [38]:
df.columns

Index(['Unnamed: 0', '房源链接', '总价', '总价单位', '单价', '小区名称', '所在大区', '所在详细区域',
       '房屋户型', '所在楼层', '建筑面积', '户型结构', '套内面积', '建筑类型', '房屋朝向', '建筑结构', '装修情况',
       '梯户比例', '配备电梯', '挂牌时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息',
       '房本备件', '房源编码', '用水类型', '用电类型', '燃气价格', '别墅类型', '室', '厅', '厨', '卫',
       '总楼层', '房屋年限/年'],
      dtype='object')

Unnamed: 0、房源链接、总价单位、小区名称、梯户比例、房源编码、抵押信息、房本备件、用水类型、用电类型、燃气价格、别墅类型，这些特征可以放弃；  
房屋户型 特征已拆分为 '室'、'厅'、'厨'、'卫'4个特征，所以房屋户型特征可以放弃；  
建筑面积和套内面积，房源销售以 建筑面积 为准，所以 套内面积 特征可以放弃；  
挂牌时间、上次交易特征、房屋年限，这3个特征已经处理为 房屋年限/年 特征，所以这3个特征可以放弃；  
总价和单价特征，我们要预测的是房屋总价，所以 单价 特征可以放弃，总价特征作为标签；  

In [39]:
# 筛选字段
df = df[['总价', '所在大区', '所在详细区域', '所在楼层', '建筑面积', '户型结构', '建筑类型', '房屋朝向', '建筑结构',
         '装修情况', '配备电梯', '交易权属', '房屋用途', '产权所属', '总楼层', '房屋年限/年', '室', '厅', '厨', '卫']]

# 特征选择 ,'房屋年限/年',空值会导致模型失败
features = ['所在大区', '所在详细区域', '所在楼层', '建筑面积', '户型结构', '建筑类型', '房屋朝向', '建筑结构',
            '装修情况', '配备电梯', '交易权属', '房屋用途', '产权所属', '总楼层', '房屋年限/年', '室', '厅', '厨', '卫']

# 标签选择
target = ['总价']

In [40]:
df.head()

,总价,所在大区,所在详细区域,所在楼层,建筑面积,户型结构,建筑类型,房屋朝向,建筑结构,装修情况,配备电梯,交易权属,房屋用途,产权所属,总楼层,房屋年限/年,室,厅,厨,卫
0,660.0,思明,莲前,低楼层,126.96,平层,板楼,东南,钢混结构,精装,有,商品房,普通住宅,共有,19,"(0.0, 1.0]",3,1,1,2
1,789.7,思明,会展中心,低楼层,149.00,错层,塔楼,东南,框架结构,精装,有,商品房,普通住宅,共有,32,nan,3,2,1,2
2,1080.0,思明,体育中心,低楼层,133.67,平层,板楼,东南,钢混结构,精装,有,商品房,普通住宅,非共有,31,nan,3,2,1,1
3,378.0,思明,莲坂,高楼层,78.66,平层,板楼,南,框架结构,简装,无,商品房,普通住宅,非共有,7,nan,3,2,1,1
4,360.0,思明,仙岳社区,中楼层,87.72,平层,板楼,南,混合结构,精装,无,商品房,普通住宅,共有,7,nan,3,1,1,1


## 数据转换

观察数据可以发现，除了建筑面积、总楼层、室、厅、厨、卫，这些特征是数值格式外，其他特征都是字符串格式，并且没有大小之分，模型将无法处理，因此我们需要把这些字符串转化成数值格式，比如‘所在楼层’字段，有低楼层、中楼层、高楼层3种取值，我们把‘所在楼层’字段分为‘所在楼层=低楼层’、‘所在楼层=中楼层’、‘所在楼层=高楼层’3个字段，数值用0或1来表示。  
我们使用 sklearn 特征选择中的 DictVectorizer 类，用它可以处理符号化的对象，将符号转成数字 0/1 进行表示。

In [41]:
# DictVectorizer 类，用它将可以处理符号化的对象，将符号转成数字 0/1 进行表示。
dvec = DictVectorizer(sparse=False)

# 特征矩阵
df_features = dvec.fit_transform(df[features].to_dict(orient='record'))

# 目标矩阵
df_target = df[target].values

In [42]:
dvec.feature_names_

['交易权属=动迁安置房',
 '交易权属=商品房',
 '交易权属=房改房',
 '交易权属=私产',
 '交易权属=经济适用房',
 '产权所属=共有',
 '产权所属=非共有',
 '卫',
 '厅',
 '厨',
 '室',
 '建筑类型=塔楼',
 '建筑类型=平房',
 '建筑类型=板塔结合',
 '建筑类型=板楼',
 '建筑结构=未知结构',
 '建筑结构=框架结构',
 '建筑结构=混合结构',
 '建筑结构=砖木结构',
 '建筑结构=砖混结构',
 '建筑结构=钢混结构',
 '建筑结构=钢结构',
 '建筑面积',
 '总楼层',
 '户型结构=复式',
 '户型结构=平层',
 '户型结构=跃层',
 '户型结构=错层',
 '房屋年限/年=(0.0, 1.0]',
 '房屋年限/年=(1.0, 2.0]',
 '房屋年限/年=(10.0, 11.0]',
 '房屋年限/年=(11.0, 12.0]',
 '房屋年限/年=(12.0, 13.0]',
 '房屋年限/年=(13.0, 14.0]',
 '房屋年限/年=(14.0, 15.0]',
 '房屋年限/年=(15.0, 16.0]',
 '房屋年限/年=(16.0, 17.0]',
 '房屋年限/年=(17.0, 18.0]',
 '房屋年限/年=(18.0, 19.0]',
 '房屋年限/年=(19.0, 20.0]',
 '房屋年限/年=(2.0, 3.0]',
 '房屋年限/年=(20.0, 21.0]',
 '房屋年限/年=(21.0, 22.0]',
 '房屋年限/年=(22.0, 23.0]',
 '房屋年限/年=(23.0, 24.0]',
 '房屋年限/年=(24.0, 25.0]',
 '房屋年限/年=(25.0, 26.0]',
 '房屋年限/年=(26.0, 27.0]',
 '房屋年限/年=(27.0, 28.0]',
 '房屋年限/年=(28.0, 29.0]',
 '房屋年限/年=(29.0, 30.0]',
 '房屋年限/年=(3.0, 4.0]',
 '房屋年限/年=(30.0, 31.0]',
 '房屋年限/年=(31.0, 32.0]',
 '房屋年限/年=(33.0, 34.0]',
 '房屋年限/年=(35.0, 36.0]',
 '房屋年限/

In [43]:
len(dvec.feature_names_)

174

## 划分训练集和测试集

In [44]:
train_x, test_x, train_y, test_y = train_test_split(
    df_features, df_target, test_size=0.3, random_state=1)

## 构建模型

### 决策树回归

In [45]:
# 使用 CART 决策树回归模型
dec_regressor = DecisionTreeRegressor(random_state=7)

# 模型训练
dec_regressor.fit(train_x, train_y)

# 模型预测
pred_y = dec_regressor.predict(test_x)

# 回归模型判定指标
mae=mean_absolute_error(test_y, pred_y)
mse = mean_squared_error(test_y, pred_y)
r2 = r2_score(test_y, pred_y)

print("决策树平均绝对误差 = {:.2f}".format(mae))
print("决策树回归均方误差 = {:.2f}".format(mse))
print("决策树回归拟合度 = {:.4f}".format(r2))

决策树平均绝对误差 = 49.44
决策树回归均方误差 = 10562.21
决策树回归拟合度 = 0.8873


决策树回归模型拟合度是 0.8873，均方误差是 10562.21。

### KNN回归

In [46]:
# 使用KNN回归模型
knn_regressor = KNeighborsRegressor()
knn_regressor.fit(train_x, train_y)
pred_y = knn_regressor.predict(test_x)
mae=mean_absolute_error(test_y, pred_y)
mse = mean_squared_error(test_y, pred_y)
r2 = r2_score(test_y, pred_y)

print("KNN 平均绝对误差 = {:.2f}".format(mae))
print("KNN 回归均方误差 = {:.2f}".format(mse))
print("KNN 回归拟合度 = {:.4f}".format(r2))

KNN 平均绝对误差 = 80.38
KNN 回归均方误差 = 30247.87
KNN 回归拟合度 = 0.6773


KNN 模型拟合度比决策树低，均方误差比决策树大。

### 线性回归

In [47]:
# 使用线性回归模型
line_regressor = LinearRegression()
line_regressor.fit(train_x, train_y)
pred_y = line_regressor.predict(test_x)
mae=mean_absolute_error(test_y, pred_y)
mse = mean_squared_error(test_y, pred_y)
r2 = r2_score(test_y, pred_y)

print("线性回归平均绝对误差 = {:.2f}".format(mae))
print("线性回归均方误差 = {:.2f}".format(mse))
print("线性回归拟合度 = {:.4f}".format(r2))

线性回归平均绝对误差 = 12207363.98
线性回归均方误差 = 328071110826258624.00
线性回归拟合度 = -3499815801419.3745


线性回归拟合度<0，模型无效

### AdaBoost 回归

In [48]:
# 使用 AdaBoost 回归模型，模型采用 CART 决策树，迭代100次
ada_regressor = AdaBoostRegressor(
    base_estimator=dec_regressor, n_estimators=100)

# 模型训练
ada_regressor.fit(train_x, train_y)

# 模型预测
pred_y = ada_regressor.predict(test_x)

# 模型判定指标
mae = mean_absolute_error(test_y, pred_y)
mse = mean_squared_error(test_y, pred_y)
r2 = r2_score(test_y, pred_y)

print("AdaBoost 平均绝对误差 = {:.2f}".format(mae))
print("AdaBoost 回归均方误差 = {:.2f}".format(mse))
print("AdaBoost 回归拟合度 = {:.4f}".format(r2))

AdaBoost 平均绝对误差 = 39.42
AdaBoost 回归均方误差 = 6665.84
AdaBoost 回归拟合度 = 0.9289


KNN 模型拟合度0.68，线性回归模型拟合度小于0，决策树模型拟合度0.887，比前两个都要高，说明决策树模型更优， AdaBoost 在决策树模型的基础上，迭代100次，模型的拟合度从 0.887 提升到 0.9289。

## 模型调优

In [52]:
# 定义函数，计算拟合度R2
def get_r2(test_y, pred_y):
    score = r2_score(test_y, pred_y)
    return score

# 决策树深度取值范围
regressor_param_grid = [{'max_depth': [x for x in range(20)]}]

# 定义函数，对模型进行 GridSearchCV 参数调优
def GridSearchCV_work(model, train_x, train_y, test_x, test_y, param_grid):
    cross_validation = KFold(10, shuffle=True)
    response = {}
    score = make_scorer(get_r2)
    gridsearch = GridSearchCV(
        estimator=model, param_grid=param_grid, scoring=score, cv=cross_validation)

    # 模型训练
    search = gridsearch.fit(train_x, train_y)
    
    # 模型预测
    pred_y = gridsearch.predict(test_x)
    
    # 模型评估
    mse = mean_squared_error(test_y, pred_y)
    r2 = get_r2(test_y, pred_y)

    print('GridSearch 最优参数：{}'.format(search.best_params_))
    print("GridSearch 均方误差 = {:.2f}".format(mse))
    print('GridSearch 最优拟合度：{:.4f}'.format(r2))

    response['predict_y'] = pred_y
    response['回归拟合度'] = r2

    return gridsearch, response


search = GridSearchCV_work(dec_regressor, train_x,
                           train_y, test_x, test_y, regressor_param_grid)[0]
search

GridSearch 最优参数：{'max_depth': 19}
GridSearch 均方误差 = 10281.91
GridSearch 最优拟合度：0.8903


GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=7),
             param_grid=[{'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                        12, 13, 14, 15, 16, 17, 18, 19]}],
             scoring=make_scorer(get_r2))

调优之后的决策树模型，拟合度从原来的 0.8873 提升到了 0.8903，说明模型预估效果更好，用最佳的决策树深度重新建模。

In [53]:
# 使用 CART 决策树回归新建模型
new_dec_regressor = DecisionTreeRegressor(max_depth=19,random_state=7)

# 重新使用 AdaBoost 回归模型，模型采用新 CART 决策树，迭代100次
new_ada_regressor = AdaBoostRegressor(
    base_estimator=new_dec_regressor, n_estimators=100)

# 模型训练
new_ada_regressor.fit(train_x, train_y)

# 模型预测
pred_y = new_ada_regressor.predict(test_x)

# 模型评估指标
mae=mean_absolute_error(test_y, pred_y)
mse = mean_squared_error(test_y, pred_y)
r2 = r2_score(test_y, pred_y)

print("AdaBoost 平均绝对误差 = {:.2f}".format(mae))
print("AdaBoost 回归均方误差 = {:.2f}".format(mse))
print("AdaBoost 回归拟合度 = {:.4f}".format(r2))

AdaBoost 平均绝对误差 = 39.40
AdaBoost 回归均方误差 = 6178.60
AdaBoost 回归拟合度 = 0.9341


升级之后的 AdaBoost 强回归模型，拟合度0.9341，比原来的 AdaBoost 回归模型拟合度 0.9289 提升了0.052，均方误差降低了487.24。

## 总结
本项目基于厦门二手房在售房源数据，构建了一个参数调优之后的 AdaBoost 回归模型，拟合度达到0.9341，模型预测效果还不错。
完整的数据挖掘流程到这里就结束了，模型还有一个可以改善的地方：  
- 特征选择部分，模型还可以有更多的特征输入，比如是否学区房、离地铁站距离、空气质量情况、附近购物中心等等，现实世界里这些特征都会对房价产生一定的影响，我们采集到的数据不包含这些特征。
- 可以采用其他更优的算法搭建模型，让模型预测的准确率更好。